In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\svpon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
import import_ipynb
from carga import doCarga
diretorio = '..\\dados\\'
dfNotas = doCarga(diretorio, opcao=3)
dfNotas = dfNotas.drop(columns=['Remetente'])

In [41]:
# Separa os contadores
count_class_0, count_class_1, count_class_2, count_class_3, count_class_4 = dfNotas['Categoria'].value_counts()

In [83]:
dfNotas

,Categoria,NovaDescricao
0,30.0simsim,CACHACA SAO FRANCISCO 12X970M
1,30.0simsim,CACHACA YPIOCA EMP.OURO
2,30.0simsim,CACHACA YPIOCA OURO
3,30.0simsim,CACHACA YPIOCA PRATA
4,30.0simsim,BB CACHACA SAGATIBA PURA (12) GF
...,...,...
99995,38.0simnão,SHAMP.INF.BEBE NATUREZA SUAVE
99996,38.0simnão,SHAMP.INF.BEBE NATUREZA SUAVE
99997,38.0simnão,SHAMP.INF.CARROS II MCQUEEN
99998,38.0simnão,SHAMP.INF.CARROS II MCQUEENNC


In [42]:
# Divide datasets por classes

df_class_0 = dfNotas[dfNotas['Categoria'] == "31.0simsim"]
df_class_1 = dfNotas[dfNotas['Categoria'] == "38.0simsim"]
df_class_2 = dfNotas[dfNotas['Categoria'] == "38.0simnão"] 
df_class_3 = dfNotas[dfNotas['Categoria'] == "30.0simsim"]         # Tamanho escolhido
df_class_4 = dfNotas[dfNotas['Categoria'] == "31.0nãonão"]         # Classse MUITO pequena - UNDERSAMPLING

# Equipara os tamanhos 30.0simsim que tem 3090 ocorrências
# Foram equiparados ao tamanho da classe 3
MaxSize = 3090 # df_class_3.size 
df_class_0 = df_class_0.sample(MaxSize)
df_class_1 = df_class_1.sample(MaxSize)
df_class_2 = df_class_2.sample(MaxSize)

# Random Oversampling
df_class_4 = df_class_4.sample(MaxSize, replace=True)

# Agrupa novamente
dfAjustado = pd.concat([df_class_0, df_class_1, df_class_2, df_class_3, df_class_4], axis=0)

In [43]:
dfAjustado = dfAjustado.sample(frac=1).reset_index(drop=True)
base_treino = dfAjustado.iloc[:10300,:]
base_teste = dfAjustado.iloc[10301:,:]

#exemplo_base_treino.Categoria.value_counts()
base_teste.shape

(5149, 2)

## Stop Words

In [6]:
lista_Stop = nltk.corpus.stopwords.words('portuguese')
#np.transpose(lista_Stop)

In [7]:
def removeStopWords(texto):
    frases = []
    for (palavras, sentimento) in texto:
        # List Comprehension para pegar palavras fora do lista_Stop
        semStop = [ p for p in palavras.split() if p not in lista_Stop]
        frases.append((semStop, sentimento))
    return frases

## Stemming

In [55]:
def aplica_Stemmer(texto):
    nltk.download('rslp')
    stemmer = nltk.stem.RSLPStemmer()
    frases_sem_Stemming = []
    for ( Categoria, NovaDescricao) in texto:
        com_Stemming = [str(stemmer.stem(p)) for p in NovaDescricao.split() if p not in lista_Stop]
        frases_sem_Stemming.append((com_Stemming, Categoria))
    return frases_sem_Stemming

In [57]:
frases_com_Stem_treinamento = aplica_Stemmer(base_treino.values)

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\svpon\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


<class 'numpy.ndarray'>


In [58]:
pd.DataFrame(frases_com_Stem_treinamento, columns=['Categoria', 'Sentimento']).sample(10)

,Categoria,Sentimento
879,"[cred, icm, propri, conf., art.330,, 1., ref.a...",31.0nãonão
8514,"[kit, sort, acessori, c/4, kit, 3un]",31.0nãonão
5146,"[vin.port.tom, duorum]",31.0simsim
6010,"[ag, ox, yam, 40v]",38.0simnão
8462,"[sh, capicilin, ojon, mono, s/]",38.0simnão
3273,"[cervej, skol, beat, sens, long]",31.0nãonão
5620,"[oe, manjerica, terr, fl]",38.0simnão
8859,"[esm, the, fusion, ros, ros]",38.0simsim
7622,"[v., port., paul, laure, class, tint, 1,5, lt.]",31.0simsim
9279,"[vin.fr.l, jamell, viogni]",31.0simsim


In [59]:
frases_com_Stem_teste = aplica_Stemmer(base_teste.values)

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\svpon\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


<class 'numpy.ndarray'>


In [62]:
def busca_Palavras(frases):
    todas_Palavras = []
    for (palavras, Categoria) in frases:
        todas_Palavras.extend(palavras)
    return todas_Palavras

In [70]:
palavras_treinamento = busca_Palavras(frases_com_Stem_treinamento)
palavras_teste = busca_Palavras(frases_com_Stem_teste)

In [65]:
print ('Quantidade de palavras no treinamento {}'.format(pd.DataFrame(palavras_treinamento).count()))

Quantidade de palavras no treinamento 0    47981
dtype: int64


In [66]:
def busca_frequencia(palavras):
    palavras = nltk.FreqDist(palavras)
    return palavras

In [67]:
frequencia_treinamento = busca_frequencia(palavras_treinamento)

In [68]:
frequencia_treinamento.most_common(20)

[('sh', 1007),
 ('cachac', 996),
 ('esm', 743),
 ('beb', 469),
 ('kit', 442),
 ('conf', 435),
 ('aguard', 367),
 ('gf', 367),
 ('cervej', 340),
 ('6x1', 323),
 ('1.', 310),
 ('tto', 309),
 ('nf', 307),
 ('dec', 300),
 ('ressarc', 298),
 ('icm', 298),
 ('skol', 279),
 ('beat', 269),
 ('vinh', 263),
 ('lt', 249)]

In [71]:
frequencia_teste = busca_frequencia(palavras_teste)

In [72]:
def busca_palavras_unicas(frequencia):
    freq = frequencia.keys()
    return freq

palavras_unicas_treinamento = busca_palavras_unicas(frequencia_treinamento)
palavras_unicas_teste = busca_palavras_unicas(frequencia_teste)

In [74]:
def extrator_palavras(documento):
    doc = set(documento)
    caracteristicas = {}
    for palavras in palavras_unicas_treinamento:
        caracteristicas['%s' % palavras] = (palavras in doc)
    return caracteristicas

In [75]:
def extrator_palavras_teste(documento):
    doc = set(documento)
    caracteristicas = {}
    for palavras in palavras_unicas_teste:
        caracteristicas['%s' % palavras] = (palavras in doc)
    return caracteristicas

In [76]:
base_completa_treinamento = nltk.classify.apply_features(extrator_palavras, frases_com_Stem_treinamento)
base_completa_teste = nltk.classify.apply_features(extrator_palavras_teste, frases_com_Stem_teste)

# Naives

In [77]:
classificador = nltk.NaiveBayesClassifier.train(base_completa_treinamento)

In [78]:
print(classificador.labels())

['30.0simsim', '31.0simsim', '31.0nãonão', '38.0simsim', '38.0simnão']


In [79]:
print(classificador.show_most_informative_features(10))

Most Informative Features
                      sh = True           38.0si : 38.0si =    403.6 : 1.0
                  aguard = True           30.0si : 31.0si =    239.7 : 1.0
                    sens = True           31.0nã : 38.0si =    127.4 : 1.0
                      lt = True           31.0nã : 38.0si =    125.4 : 1.0
                     art = True           31.0nã : 30.0si =    118.7 : 1.0
                      v. = True           31.0si : 38.0si =    110.8 : 1.0
                     cop = True           31.0nã : 38.0si =    105.2 : 1.0
                     bco = True           31.0si : 38.0si =     91.5 : 1.0
                      bl = True           38.0si : 31.0si =     88.8 : 1.0
                      ag = True           30.0si : 38.0si =     78.6 : 1.0
None


In [80]:
print (nltk.classify.accuracy(classificador, base_completa_teste))

0.9728102544183337


In [81]:
erros = []
for (frase, Categoria) in base_completa_teste:
    resultado = classificador.classify(frase)
    if resultado != Categoria:
        erros.append((Categoria, resultado, frase))

In [82]:
from nltk.metrics import ConfusionMatrix
esperado = []
previsto = []
for (frase, Categoria) in base_completa_teste:
    resultado = classificador.classify(frase)
    previsto.append(resultado)
    esperado.append(Categoria)

matriz = ConfusionMatrix(esperado, previsto)
print (matriz)

           |    3    3    3    3    3 |
           |    0    1    1    8    8 |
           |    .    .    .    .    . |
           |    0    0    0    0    0 |
           |    s    n    s    s    s |
           |    i    ã    i    i    i |
           |    m    o    m    m    m |
           |    s    n    s    n    s |
           |    i    ã    i    ã    i |
           |    m    o    m    o    m |
-----------+--------------------------+
30.0simsim | <995>   .    3    1    . |
31.0nãonão |   13<1003>   .    .    . |
31.0simsim |   39    4 <990>   2    4 |
38.0simnão |    8    .    5<1034>   5 |
38.0simsim |   13    1    8   34 <987>|
-----------+--------------------------+
(row = reference; col = test)

